<a href="https://colab.research.google.com/github/nishantdhiman30/Object-Recognition-and-Detection/blob/main/Create_Annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect google drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
#Import Libraries
import pandas as pd
import os

In [ ]:
#Read CSV File
os.chdir('/content/drive/My Drive/Object_detection_and_recognition/Dataset_and_Annotations/csv_folder')
classes_data = pd.read_csv('class-descriptions-boxable.csv', header=None)
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [ ]:
#[Generating Unique Identifiers]


#Define required classes
classes = ['Cat', 'Dog']

class_strings = []
#Get the strings corresponding to the class name and store it inside a list called as class_strings
for i in classes:
  #Extract the first column's elements for matching classes
  req_classes = classes_data.loc[classes_data[1] == i]
  string = req_classes.iloc[0][0]
  print(string)
  #Append the strings corresponding to the class name into the list
  class_strings.append(string) 

print(class_strings)


/m/01yrx
/m/0bt9lr
['/m/01yrx', '/m/0bt9lr']


In [ ]:
#Get only required columns from annotations csv file and get rid of NOT required columns
annotation_data = pd.read_csv('train-annotations-bbox.csv', 
                              usecols=['ImageID', 
                                       'LabelName',
                                       'XMin',
                                       'XMax',
                                       'YMin',
                                       'YMax'])
annotation_data.head()

In [ ]:
#Get only records with matching classes (LabelName = class_strings)
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_strings)].copy()
filtered_class_data.head()

In [ ]:
#Add new columns that are required for YOLO format
filtered_class_data['classNumber'] = ''
filtered_class_data['center x'] = ''
filtered_class_data['center y'] = ''
filtered_class_data['width'] = ''
filtered_class_data['height'] = ''


In [ ]:
#Iterate through all the class strings and assign a class number according to the order they are appearing in the list
for i in range(len(class_strings)):
  filtered_class_data.loc[filtered_class_data['LabelName'] == class_strings[i], 'classNumber'] = i

#Calculate center x and center y values
filtered_class_data['center x'] = (filtered_class_data['XMax'] + filtered_class_data['XMin']) / 2
filtered_class_data['center y'] = (filtered_class_data['YMax'] + filtered_class_data['YMin']) / 2

#Calculate width and height values
filtered_class_data['width'] = filtered_class_data['XMax'] - filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax'] - filtered_class_data['YMin']

#Generate the dataframe with YOLO required values
YOLO_values = filtered_class_data.loc[:, ['ImageID','classNumber','center x','center y','width','height']].copy()
YOLO_values.head()

In [ ]:
#Change the current directory to the one where we have Images
image_path = '/content/drive/My Drive/Object_detection_and_recognition/Dataset_and_Annotations/Dataset/train/Cat_Dog'
os.chdir(image_path)

#Go through all the image files in the directory 
#Fullstop in os.walk('.') means the current directory
for current_dir, dirs, files in os.walk('.'):
  for f in files:
    #Check if the files extension ends with '.jpg'
    if f.endswith('.jpg'):
      #Extract only the title of the image file so that corresponding txt file can be generated with same name 
      #As per YOLO guidelines
      image_title = f[:-4]
      #Get matched data
      YOLO_file = YOLO_values.loc[YOLO_values['ImageID'] == image_title]

      #Create a copy of the above dataframe to avoid any mess
      df = YOLO_file.loc[:, ['classNumber', 'center x', 'center y', 'width', 'height']].copy()

      save_path = image_path + '/' + image_title + '.txt'

      #Generate a text_file containing required data in YOLO format
      df.to_csv(save_path, header=False, index=False, sep=' ')